<center><b>In the name of God</b></center>

<b>Course</b>: Machine Learning
<br>
<b>Description:</b> HomeWork 1 | Question 3
<br>
<b>Developer</b>: Alireza Mazochi (400131075)

# Libraries

In [35]:
# Import Allowed Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load

In [36]:
train_data = pd.read_csv("train.csv")

In [37]:
test_data = pd.read_csv("test.csv")

In [39]:
set(train_data["region"].values)

{'northeast', 'northwest', 'southeast', 'southwest'}

# Preprocessing

In [40]:
# Encoding string features

def gender_encoding(gender):
    if gender == "male":
        return 1
    elif gender == "female":
        return 0
    else:
        raise Exception("An expected value for gender!")
    
def smoker_encoding(smoker):
    if smoker == "yes":
        return 1
    elif smoker == "no":
        return 0
    else:
        raise Exception("An expected value for smoker!")

def region_encoding(region):
    if region == "northeast":
        return np.array([1, 0, 0, 0])
    elif region == "northwest":
        return np.array([0, 1, 0, 0])
    elif region == "southeast":
        return np.array([0, 0, 1, 0])
    elif region == "southwest":
        return np.array([0, 0, 0, 1])
    else:
        raise Exception("An expected value for region!")

In [41]:
# Apply encodings

train_data["gender"] = train_data["gender"].apply(gender_encoding)
train_data["smoker"] = train_data["smoker"].apply(smoker_encoding)
train_data["region"] = train_data["region"].apply(region_encoding)

test_data["gender"] = test_data["gender"].apply(gender_encoding)
test_data["smoker"] = test_data["smoker"].apply(smoker_encoding)
test_data["region"] = test_data["region"].apply(region_encoding)

In [42]:
def convert_vector_feature_to_separated_features(data):
    data["region-0"] = data["region"].apply(lambda x:x[0])
    data["region-1"] = data["region"].apply(lambda x:x[1])
    data["region-2"] = data["region"].apply(lambda x:x[2])
    data["region-3"] = data["region"].apply(lambda x:x[3])
    del data["region"]

In [43]:
convert_vector_feature_to_separated_features(train_data)
convert_vector_feature_to_separated_features(test_data)

In [45]:
test_data

,age,gender,bmi,children,smoker,charges,region-0,region-1,region-2,region-3
0,30,1,22.990,2,1,17361.76610,0,1,0,0
1,24,1,32.700,0,1,34472.84100,0,0,0,1
2,24,1,25.800,0,0,1972.95000,0,0,0,1
3,48,1,29.600,0,0,21232.18226,0,0,0,1
4,47,1,19.190,1,0,8627.54110,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
145,52,1,32.775,3,0,11289.10925,0,1,0,0
146,60,1,32.800,0,1,52590.82939,0,0,0,1
147,20,0,31.920,0,0,2261.56880,0,1,0,0
148,55,1,21.500,1,0,10791.96000,0,0,0,1


# Basis Function

In [46]:
train_data["age"] = train_data["age"].apply(lambda x:x**2)
test_data["age"] = test_data["age"].apply(lambda x:x**2)

# X/Y split

In [48]:
x_train = train_data.drop(columns =["charges"]).values
y_train = train_data["charges"].values

x_test = test_data.drop(columns = ["charges"]).values
y_test = test_data["charges"].values

# Multuvariate

In [49]:
class Multivariate():
    """ Class Polynomial store coefficients of a multivariate"""

    def __init__(self, number_variables):
        """ 
        Create a polynomial object.
        
        number_variables: number of variables
        """
        
        self.number_variables = number_variables

        # Coefficients of multivariate: a[n-1]*x[n-1] + a[n-2]*x[n-2] + ... + a[1]*(x[1]) + a[0]*x[0] + c
        self.a = np.zeros(number_variables)
        self.c = 0
        
    def single_apply(self, x):
        """
        Apply the multivariate to a point
        
        x: a point
        """
        return np.dot(x, self.a) + self.c
    
    def apply(self, x):
        """ 
        Apply the multivariate to an array of points
        
        x: array of points
        """
        return np.array(self.single_apply(x))
    
    
    def set_params(self, a, c):
        """ Setter for params"""
        self.a = a
        self.c = c
    
    def get_params(self):
        """ Getter for params"""
        return self.a, self.c
    
    def __str__(self):
        """ String of the multivariate """
        variables_name = list(data.drop(columns =["charges"]).columns)
        
        output = "charges = "
        for i in range(self.number_variables):
            output += "{:.2f}".format(self.a[i]) + " × " + variables_name[i] + " + "
        
        output += "{:.2f}".format(self.c)
        return output

# Loss Fuction

In [50]:
class LossFunction():
    """ Some Loss functions and partial derivation of them located in this class """

    @staticmethod
    def mse(y_predict, y_real):
        """ Mean Square Error """
        arr = (np.subtract(y_real,y_predict))**2
        return np.sum(arr) / len(y_predict)
    
    @staticmethod
    def partial_mse_a(y_predict, y_real, x, index):
        """ Partial derivation of Mean Square Error for coeficient of feature of multivariat"""
        arr = np.multiply(x[:, index], (np.subtract(y_real, y_predict)))    
        return -2*np.sum(arr)/len(x)
    
    @staticmethod
    def partial_mse_c(y_predict, y_real):
        """ Partial derivation of Mean Square Error for consist coefficient of multivariat"""
        arr = np.subtract(y_real, y_predict) 
        return -2*np.sum(arr)/len(y_real)


# Gradient Descent

In [51]:
class MultivariateGradientDescent():
    """ This class try to match a pattern to train data with Gradient Descent method """
        
    def __init__(self, number_epoch, number_variables):
        """
        Create a MultivariateGradientDescent
        
        number_epoch: The required number of epochs
        number_variables: The number of variables
        
        """
        
        # Learning rate configuration
        self.learning_rate_initial = 1
        self.learning_rate = self.learning_rate_initial
        self.decrease_learning_rate = self.learning_rate_initial / number_epoch
            
        # Array for storing error and learning values 
        self.train_error = []
        self.test_error = []
        self.learning_rate_value = []
    
        self.number_epoch = number_epoch
        
        self.multivariate = Multivariate(number_variables=number_variables)
        
        
    def run(self):
        """ Run Gradient Descent! """
        for epoch in range(self.number_epoch):
            self._collect_epoch_statistic(epoch)
            
            a_new = []
            y_predict = self.multivariate.apply(x_train)
            
            a_cur, c_cur = self.multivariate.get_params()
            
            for i, ai in enumerate(a_cur):
                ai_new = ai - self.learning_rate * LossFunction.partial_mse_a(y_predict, y_train, x_train, i)
                a_new.append(ai_new)
            
            c_new = LossFunction.partial_mse_c(y_predict, y_train)
                
            self.multivariate.set_params(np.array(a_new), c_new)
            self.learning_rate -= self.decrease_learning_rate
            
        self._collect_epoch_statistic(epoch)
            
    def _collect_epoch_statistic(self, epoch):
        """ Collect statistics per epoch """
        self.learning_rate_value.append(self.learning_rate)
        
        y_predict = self.multivariate.apply(x_train)
        self.train_error.append(LossFunction.mse(y_predict, y_train))
        
        y_predict = self.multivariate.apply(x_test)
        self.test_error.append(LossFunction.mse(y_predict, y_test))
    
    def plot_result(self):
        """ Plot train error and test error over epochs and predicted equation """
        x = np.arange(0, self.number_epoch+1)
        
        plt.plot(x, self.train_error, color="green", label="Train Error")
        plt.plot(x, self.test_error, color="red", label="Test Error")
        plt.legend()
        plt.title(f"Train Error VS Test Error \n{self._configuration_title()}")
        
        plt.xlabel("Epoch")
        plt.ylabel("Error")
        plt.show()
    
        # Predicted pattern
        print(self.multivariate)
        
        # Final Error Value
        y_predict = self.multivariate.apply(x_train)
        train_error = LossFunction.mse(y_predict, y_train)
        
        y_predict = self.multivariate.apply(x_test)
        test_error = LossFunction.mse(y_predict, y_test)
        
        print()
        print(f"Final Train Error= {train_error}")
        print(f"Final Test Error= {test_error}")
        
    
    def plot_learning_rate_value(self):
        """ Plot changing of the learning rate over epochs """
        x = np.arange(0, self.number_epoch+1)
        
        plt.plot(x, self.learning_rate_value, color = "yellow")
        plt.xlabel("Epoch")
        plt.ylabel("Learning Rate")
        plt.title(f"Learning Rete Changes \n {self._configuration_title()}")
        plt.show()
        
    def _configuration_title(self):
        """ Generating a string that contains configurations of the method """
        return f"Number of Features = {self.multivariate.number_variables} | Number Epochs = {self.number_epoch}"
    
    

In [52]:
multivariate_gradient_descent = MultivariateGradientDescent(number_epoch = 5000, number_variables = len(x_train[0]))

In [53]:
multivariate_gradient_descent.run()

NameError: name 'c' is not defined